In [6]:
import os
import json
import autogen
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from collections import Counter

In [7]:
DATA_PATH = Path("/home/kotov-d@ad.speechpro.com/projects/AGENTS/data/result.csv")
df = pd.read_csv(DATA_PATH)
df = df[df["file_path"] == '/mnt/cs/nlu/home/kotov/LLM/auto_validation/data_from_stc_cloud/mosmetro_hand_tests_semantic.xlsx']
df.shape

(31, 30)

In [8]:
df.columns

Index(['Кейс', 'Вопрос пользователя', 'Референсный ответ', 'Чанк 1', 'Чанк 2',
       'Чанк 3', 'Чанк 4', 'Ответ модели', 'Качество поиска',
       'Понимание задания', 'Соответствие док-там', 'Непротиворечивость',
       'Полнота', 'Итог', 'Разметчик', 'Комментарий', 'file_path',
       'Референсный документ', 'Референсный документ 2',
       'helpsteer-helpfulness', 'helpsteer-correctness', 'helpsteer-coherence',
       'helpsteer-complexity', 'helpsteer-verbosity',
       'giga_neprotivirechivost', 'giga_polnota', 'gpt4o_neprotivirechivost',
       'gpt4o_polnota', 'llama70b_neprotivorechivost', 'llama70b_polnota'],
      dtype='object')

In [9]:
local_llm_config = {
    "config_list": [
        {
            "model": "/mnt/hs/dorado6/models/meta/llama-3-70b-instruct", # Sam2,e as in vLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:8000/v1"  # Your vLLM URL, with '/v1' added
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

llm_config = {"config_list": [{"model": "gpt-4o", "api_key": OPENAI_API_KEY}], "cache_seed": 42}

In [10]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },  
    human_input_mode="TERMINATE",
)
spk1 = autogen.AssistantAgent(
    name="Speaker1",
    system_message="Эксперт в российской железнодорожной индустрии",
    llm_config=llm_config,
)
spk2 = autogen.AssistantAgent(
    name="Speaker2",
    system_message="Эксперт в российской железнодорожной индустрии. Отвечай только по-русски",
    llm_config=local_llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, spk1, spk2], messages=[], max_round=3)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
# chat_result = user_proxy.initiate_chat(
#     manager, message=f"""Непротиворечит ли ответ модели правильному ответу? Порассуждайте с коллегами на эту тему, аргументируя свое мнение, и выведите результат вашей дискуссии. 0 если ответ модели полностью противоречит референсному ответу, 1- если ответ модели частично противоречит референсному ответу и здравому смыслу, 2- результат не противоречит референсному и здравому смыслуответу  \n**Вопрос**\n{df.loc[148]['Вопрос пользователя']}\n\n**Ответ модели:**\n{df.loc[148]['Ответ модели']}\n\n**Референсный ответ:**\n{df.loc[148]['Референсный ответ']}\n"""
# )
# # type exit to terminate the chat

In [18]:
df.shape

(31, 30)

In [4]:
df.columns

Index(['Кейс', 'Вопрос пользователя', 'Референсный ответ', 'Чанк 1', 'Чанк 2',
       'Чанк 3', 'Чанк 4', 'Ответ модели', 'Качество поиска',
       'Понимание задания', 'Соответствие док-там', 'Непротиворечивость',
       'Полнота', 'Итог', 'Разметчик', 'Комментарий', 'file_path',
       'Референсный документ', 'Референсный документ 2',
       'helpsteer-helpfulness', 'helpsteer-correctness', 'helpsteer-coherence',
       'helpsteer-complexity', 'helpsteer-verbosity',
       'giga_neprotivirechivost', 'giga_polnota', 'gpt4o_neprotivirechivost',
       'gpt4o_polnota', 'llama70b_neprotivorechivost', 'llama70b_polnota'],
      dtype='object')

In [ ]:
summaries, histories = [], []

for idx, row in tqdm(df.iterrows()):
    try:
        chat_result = user_proxy.initiate_chat(
            manager, message=f"""Непротиворечит ли ответ модели правильному ответу? Порассуждайте с коллегами на эту тему, аргументируя свое мнение, и выведите результат вашей дискуссии. 0 если ответ модели полностью противоречит референсному ответу, 1- если ответ модели частично противоречит референсному ответу или здравому смыслу, 2- результат не противоречит референсному ответу или здравому смыслу  \n**Вопрос**\n{df.loc[idx]['Вопрос пользователя']}\n\n**Ответ модели:**\n{df.loc[idx]['Ответ модели']}\n\n**Референсный ответ:**\n{df.loc[idx]['Референсный ответ']}\n"""
        )
        summaries.append(chat_result.summary)
        histories.append(json.dumps(chat_result.chat_history, indent=2, ensure_ascii=False))
    except:
        print(idx)

In [13]:
len(summaries)
# df.iloc[28:].shape

30

In [35]:
agent_results = df.drop(index=138).copy()
agent_results["agent_summary"] = summaries
agent_results["agent_history"] = histories
agent_results.shape

(30, 32)

In [36]:
def get_score(tail):
    if '0' in tail:
        return 0
    elif '2' in tail:
        return 2
    elif '1' in tail:
        return 1
    else:
        return None
    
def in_tail(tail):
    to_return = set()
    if '0' in tail:
        to_return.add('0')
    if '1' in tail:
        to_return.add('1')
    if '2' in tail:
        to_return.add('2')
    return to_return

In [37]:
agent_res = []

for i in agent_results["agent_summary"]:
    tail = i[-200:]
    
    if len(in_tail(tail).intersection(set(['0','1','2']))) > 1:
        tail = tail[-100:]
        if len(in_tail(tail).intersection(set(['0','1','2']))) > 1:
            score = get_score(tail[-50:])
        else:
            score = get_score(tail)
    else:
        score = get_score(tail)

    agent_res.append(score)
        
Counter(agent_res)

Counter({1: 12, None: 6, 0: 6, 2: 6})

In [46]:
# agent_results["agent_score"] = agent_res
# len(agent_res)

31

In [38]:
agent_results.columns

Index(['Кейс', 'Вопрос пользователя', 'Референсный ответ', 'Чанк 1', 'Чанк 2',
       'Чанк 3', 'Чанк 4', 'Ответ модели', 'Качество поиска',
       'Понимание задания', 'Соответствие док-там', 'Непротиворечивость',
       'Полнота', 'Итог', 'Разметчик', 'Комментарий', 'file_path',
       'Референсный документ', 'Референсный документ 2',
       'helpsteer-helpfulness', 'helpsteer-correctness', 'helpsteer-coherence',
       'helpsteer-complexity', 'helpsteer-verbosity',
       'giga_neprotivirechivost', 'giga_polnota', 'gpt4o_neprotivirechivost',
       'gpt4o_polnota', 'llama70b_neprotivorechivost', 'llama70b_polnota',
       'agent_summary', 'agent_history'],
      dtype='object')

In [39]:
CSV_PATH = Path("/home/kotov-d@ad.speechpro.com/projects/AGENTS/data/agent_result_gpt_llama.csv")
EXCEL_PATH = Path("/home/kotov-d@ad.speechpro.com/projects/AGENTS/data/agent_result_gpt_llama.xlsx")

In [40]:
agent_results.to_csv(CSV_PATH, index=False)

In [41]:
# agent_results[agent_results["agent_score"].isna()]

In [ ]:
# json_string = agent_results[agent_results["agent_score"].isna()].iloc[0,:]["agent_history"]

In [14]:
# json.loads(json_string)

[{'content': 'Непротиворечит ли ответ модели правильному ответу? Порассуждайте с коллегой на эту тему и выведите результат вашей дискуссии. Ответ должен быть строго в формате "Ответ: [VALUE]", где [VALUE], это 0 если ответ модели полностью противоречит референсному ответу, 1- если ответ модели частично противоречит референсному ответу и здравому смыслу, 2- результат не противоречит референсному и здравому смыслуответу  \n**Вопрос**\nМне нужно съездить на метро до ВДНХ и обратно. Выгодно ли брать «единый» на сутки?\n\n**Ответ модели:**\nЕсли вы планируете поездку до ВДНХ и обратно, то выгоднее взять единый на сутки. Два билета стоят 124 рубля, а единый на сутки стоит 285 рублей. Однако, если вы собираетесь совершить больше двух поездок или планируете использовать другие виды транспорта, то может быть выгоднее приобрести другие типы билетов или использовать карту "Тройка". Подробнее о тарифах и правилах проезда можно узнать на официальном сайте Московского Метро https://mosmetro.ru/passe

In [15]:
# agent_results.loc[7, "agent_score"] = 2

In [32]:
# agent_results = agent_results.fillna("None")
# agent_results.to_csv(CSV_PATH, index=False)

# ====================================

In [42]:
agent_results = pd.read_csv(CSV_PATH)

sub_df = agent_results.loc[:, ["agent_history",'Непротиворечивость', 'gpt4o_neprotivirechivost']]
sub_df.shape

(30, 3)

In [43]:
qs, llamas, gpts = [], [], []
for json_string in sub_df["agent_history"]:
    cur_data = json.loads(json_string)
    qs.append(cur_data[0]['content'])
    gpt_ans = cur_data[1]['content']
    gpts.append(gpt_ans)
    llama_ans = cur_data[2]['content']
    llamas.append(llama_ans)

In [45]:
new_df = pd.DataFrame(columns=["question", "gpt_1_turn", "llama_2_turn"])
new_df["question"] = qs
new_df["gpt_1_turn"] = gpts
new_df["llama_2_turn"] = llamas

new_df['agent_score'] = None
new_df['Непротиворечивость'] = agent_results.loc[:, ['Непротиворечивость']]
new_df['gpt4o_neprotivirechivost'] = agent_results.loc[:, ['gpt4o_neprotivirechivost']]

new_df = new_df.fillna("None")

In [46]:
import xlsxwriter
workbook = xlsxwriter.Workbook(EXCEL_PATH)
worksheet = workbook.add_worksheet(f"вопрос {1}")

worksheet.set_column('A:A', 80)  
worksheet.set_column('B:B', 80)  
worksheet.set_column('C:C', 80)  
worksheet.set_column('D:D', 10)  
worksheet.set_column('E:E', 10)  
worksheet.set_column('F:F', 10) 

0

In [47]:
border_format = workbook.add_format({
            'border': 1,  # Add a border
            'align': 'left',  # Align text to the left
            'valign': 'top',  # Center text vertically
            'text_wrap': True  # Enable text wrapping
        })


In [48]:
for row_num, row in tqdm(new_df.iterrows()):
    worksheet.write_row(row_num, 0, row, border_format)
workbook.close()

30it [00:00, 11082.36it/s]
